In [1]:
import json


In [2]:
def find_ports(ast_node, inputs, outputs, inouts):
    if isinstance(ast_node, dict):
        if ast_node.get('tag') == 'kPortDeclaration':
            port_type = ast_node['children'][0]['tag']
            for port in ast_node['children'][1:]:
                if port_type == 'kInputDeclaration':
                    inputs.append(port['children'][0]['text'])
                elif port_type == 'kOutputDeclaration':
                    outputs.append(port['children'][0]['text'])
                elif port_type == 'kInOutDeclaration':
                    inouts.append(port['children'][0]['text'])
        for key, value in ast_node.items():
            find_ports(value, inputs, outputs, inouts)
    elif isinstance(ast_node, list):
        for item in ast_node:
            find_ports(item, inputs, outputs, inouts)


In [3]:
def find_instances(ast_node, instances):
    if isinstance(ast_node, dict):
        if ast_node.get('tag') == 'kDataDeclaration':
            for child in ast_node.get('children', []):
                if 'kModuleInstance' in child.get('tag', ''):
                    instance_name = child['children'][1]['children'][0]['text']
                    instances.append(instance_name)
        for key, value in ast_node.items():
            find_instances(value, instances)
    elif isinstance(ast_node, list):
        for item in ast_node:
            find_instances(item, instances)


In [4]:
def find_flops_and_combi_signals(ast_node, flops, combi_signals):
    if isinstance(ast_node, dict):
        tag = ast_node.get('tag')
        if tag == 'kVariableDeclarationAssignment':
            for child in ast_node.get('children', []):
                if child.get('tag') == 'kRegisterVariable':
                    for signal in child.get('children', []):
                        if signal.get('tag') == 'Symbol' and signal.get('text').startswith('r_'):
                            flops.append(signal.get('text'))
                if child.get('tag') == 'kLogicVariable':
                    for signal in child.get('children', []):
                        if signal.get('tag') == 'Symbol' and signal.get('text').startswith('w_'):
                            combi_signals.append(signal.get('text'))
        for key, value in ast_node.items():
            find_flops_and_combi_signals(value, flops, combi_signals)
    elif isinstance(ast_node, list):
        for item in ast_node:
            find_flops_and_combi_signals(item, flops, combi_signals)


In [6]:
# Read the AST from the JSON file
with open("math_adder_half.sv.ast.json", "r") as file:
    ast = json.load(file)
print(ast)
# Initialize lists
inputs = []
outputs = []
inouts = []
instances = []
flops = []
combi_signals = []

# Find ports, instances, flops, and combinatorial signals
find_ports(ast, inputs, outputs, inouts)
find_instances(ast, instances)
find_flops_and_combi_signals(ast, flops, combi_signals)

# Print results
print("Inputs:", inputs)
print("Outputs:", outputs)
print("Inouts:", inouts)
print("Instances:", instances)
print("Flops:", flops)
print("Combinatorial Signals:", combi_signals)


{'math_adder_half.sv': {'tree': {'children': [{'children': [{'end': 10, 'start': 0, 'tag': '`timescale'}, {'children': [{'end': 14, 'start': 11, 'tag': 'TK_TimeLiteral', 'text': '1ns'}], 'tag': 'kTimeLiteral'}, {'end': 16, 'start': 15, 'tag': '/'}, {'children': [{'end': 20, 'start': 17, 'tag': 'TK_TimeLiteral', 'text': '1ps'}], 'tag': 'kTimeLiteral'}], 'tag': 'kTimescaleDirective'}, {'children': [{'children': [{'end': 28, 'start': 22, 'tag': 'module'}, None, {'end': 44, 'start': 29, 'tag': 'SymbolIdentifier', 'text': 'math_adder_half'}, None, None, {'children': [{'end': 46, 'start': 45, 'tag': '('}, {'children': [{'children': [{'end': 56, 'start': 51, 'tag': 'input'}, None, {'children': [None, None, None, None], 'tag': 'kDataType'}, {'children': [{'end': 61, 'start': 58, 'tag': 'SymbolIdentifier', 'text': 'i_a'}, None], 'tag': 'kUnqualifiedId'}, {'children': [None], 'tag': 'kUnpackedDimensions'}, None], 'tag': 'kPortDeclaration'}, {'end': 62, 'start': 61, 'tag': ','}, {'children': [{'c